In [ ]:
import pandas as pd
import numpy as np
# transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# nlp
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
!apt install -qq enchant
!pip install pyenchant
import enchant # pip install pyenchant
import spacy

# Utility functions

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [ ]:
def split_segment_words(chars, exclude=None, dictionary = enchant.Dict("en_US")):
    """
    Segment a string of chars using the pyenchant vocabulary.
    Keeps longest possible words that account for all characters,
    and returns list of segmented words.

    :param chars: (str) The character string to segment.
    :param exclude: (set) A set of string to exclude from consideration.
                    (These have been found previously to lead to dead ends.)
                    If an excluded word occurs later in the string, this
                    function will fail.
    """
    words = []

    if not chars.isalpha():  # don't check punctuation etc.; needs more work
        return [chars]

    if not exclude:
        exclude = set()

    working_chars = chars
    while working_chars:
        # iterate through segments of the chars starting with the longest segment possible
        for i in range(len(working_chars), 1, -1):
            segment = working_chars[:i]
            if dictionary.check(segment) and segment not in exclude:
                words.append(segment)
                working_chars = working_chars[i:]
                break
        else:  # no matching segments were found
            if words:
                exclude.add(words[-1])
                return split_segment_words(chars, exclude=exclude)
            # let the user know a word was missing from the dictionary,
            # but keep the word
            # print('"{chars}" not in dictionary (so just keeping as one segment)!'
            #       .format(chars=chars))
            return [chars]
    # return a list of words based on the segmentation
    return words

In [ ]:
def split_words(str_arr):
  list_split_words = []
  for chars in str_arr:
    split_word = split_segment_words(chars)
    list_split_words.extend(split_word)
  return list_split_words

# processing job posts

In [ ]:
url = dict()
url['wa'] = 'https://drive.google.com/file/d/1m6hzBdFLamz_Ubu7JMOVgJfzsI_M8Lob/view?usp=sharing'
url['ta'] = 'https://drive.google.com/file/d/1f5e8GNPvCBCbJ0y-RNhpYWBShYt2EDa4/view?usp=sharing'
url['sf'] = 'https://drive.google.com/file/d/1ONn8TzpkPw2qHVuyhfuz6h7jquWetM71/view?usp=sharing'
url['ny'] = 'https://drive.google.com/file/d/1meTBTTj7Z9G8fYo29DnMUaJkYmwrCGnJ/view?usp=sharing'

df_jobs = []
for city in url:
  print(city)
  url[city] = 'https://drive.google.com/uc?id=' + url[city].split('/')[-2]
  url[city] = pd.read_csv(url[city])
  df_jobs.append(url[city])

df_jobs = pd.concat(df_jobs)
df_jobs = df_jobs.rename(columns={'Job_title' : 'title', 
                                  'Job_Desc' : 'description',
                                  'Industry' : 'department',
                                  'Job_Type' : 'employment_type'})


wa
ta
sf
ny


### data value counts

In [ ]:
df_jobs

,title,Company,State,City,Min_Salary,Max_Salary,description,department,Rating,Date_Posted,Valid_until,employment_type
0,Data Scientist,ManTech,VA,Chantilly,108486,145165,This company is in a hiring surge in response ...,Business Services,4.1,2020-04-30,2020-06-06,FULL_TIME
1,Data Scientist,GEICO,MD,Chevy Chase,69285,113337,GEICO’s Data Science team uses predictive anal...,Insurance,3.3,2020-05-02,2020-06-06,FULL_TIME
2,Data Scientist,Tecolote Research,DC,Washington,74791,102528,"\nData Scientist\n \n Location: Washington, D...",Aerospace & Defense,4.0,2020-05-01,2020-06-06,FULL_TIME
3,Data Scientist,Systems &amp;amp; Technology Research,VA,Arlington,94721,115685,STR is a government research contractor specia...,Aerospace & Defense,4.7,2020-04-25,2020-06-06,FULL_TIME
4,Chief Data Scientist,ManTech,VA,Alexandria,-1,-1,This company is in a hiring surge in response ...,Business Services,4.1,2020-05-06,2020-06-06,FULL_TIME
...,...,...,...,...,...,...,...,...,...,...,...,...
895,RN / LPN,Star Pediatric Home Care Agency,NJ,Jersey City,48000,75000,Owned and Operated by Nurses.\nAt Star Pediatr...,Health Care,4.5,2020-04-28,2020-06-07,FULL_TIME
896,Ultrasound Technologist,Middle Village Radiology,NY,Rego Park,55069,74745,Medical Imaging Office in Queens is seeking a ...,NaN,3.0,2020-05-07,2020-06-07,FULL_TIME
897,NYS Licensed Psychologist,The Senior Citizen Care Group,NY,Bronx,-1,-1,Licensed NYS Psychologist needed for Bronx nur...,NaN,NaN,2020-04-29,2020-06-07,FULL_TIME
898,Lead Pharmacy Technician: Billing,Medly Pharmacy,NY,Brooklyn,-1,-1,This company is in a hiring surge in response ...,Health Care,4.0,2020-05-05,2020-06-07,FULL_TIME


### stop words → lemmatization → stem

In [ ]:
df_jobs_trimmed = df_jobs[['title','department','description', 'employment_type']]
df_jobs_trimmed

,title,department,description,employment_type
0,Data Scientist,Business Services,This company is in a hiring surge in response ...,FULL_TIME
1,Data Scientist,Insurance,GEICO’s Data Science team uses predictive anal...,FULL_TIME
2,Data Scientist,Aerospace & Defense,"\nData Scientist\n \n Location: Washington, D...",FULL_TIME
3,Data Scientist,Aerospace & Defense,STR is a government research contractor specia...,FULL_TIME
4,Chief Data Scientist,Business Services,This company is in a hiring surge in response ...,FULL_TIME
...,...,...,...,...
895,RN / LPN,Health Care,Owned and Operated by Nurses.\nAt Star Pediatr...,FULL_TIME
896,Ultrasound Technologist,NaN,Medical Imaging Office in Queens is seeking a ...,FULL_TIME
897,NYS Licensed Psychologist,NaN,Licensed NYS Psychologist needed for Bronx nur...,FULL_TIME
898,Lead Pharmacy Technician: Billing,Health Care,This company is in a hiring surge in response ...,FULL_TIME


In [ ]:
df_jobs_trimmed = df_jobs_trimmed.apply(lambda x: x.astype(str).str.lower())

In [ ]:
# Load the en_core_web_sm model
nlp = spacy.load('en_core_web_sm')
stopwords = spacy.lang.en.stop_words.STOP_WORDS
# lemmatize job description
tokenized_description = []
for description in df_jobs_trimmed['description']: 
    lemmas = [token.lemma_ for token in nlp(description)]
    tokenized_description.append([lemma.lower() for lemma in lemmas if lemma.isalpha() and lemma not in stopwords or lemma == '-PRON-'])
df_jobs_trimmed['description_cleaned'] = tokenized_description

In [ ]:
df_jobs_trimmed

,title,department,description,employment_type,description_cleaned
0,data scientist,business services,this company is in a hiring surge in response ...,full_time,"[company, hire, surge, response]"
1,data scientist,insurance,geico’s data science team uses predictive anal...,full_time,"[geico, datum, science, team, use, predictive,..."
2,data scientist,aerospace & defense,"\ndata scientist\n \n location: washington, d...",full_time,"[datum, scientist, location, washington, dc, e..."
3,data scientist,aerospace & defense,str is a government research contractor specia...,full_time,"[str, government, research, contractor, specia..."
4,chief data scientist,business services,this company is in a hiring surge in response ...,full_time,"[company, hire, surge, response]"
...,...,...,...,...,...
895,rn / lpn,health care,owned and operated by nurses.\nat star pediatr...,full_time,"[operate, nurse, star, pediatric, home, care, ..."
896,ultrasound technologist,nan,medical imaging office in queens is seeking a ...,full_time,"[medical, imaging, office, queens, seek, time,..."
897,nys licensed psychologist,nan,licensed nys psychologist needed for bronx nur...,full_time,"[license, nys, psychologist, need, bronx, nurs..."
898,lead pharmacy technician: billing,health care,this company is in a hiring surge in response ...,full_time,"[company, hire, surge, response]"


In [ ]:
df_jobs_trimmed.reset_index(inplace = True)
df_jobs_trimmed.drop('index', axis=1, inplace=True)
df_jobs_trimmed

,title,department,description,employment_type,description_cleaned
0,data scientist,business services,this company is in a hiring surge in response ...,full_time,"[company, hire, surge, response]"
1,data scientist,insurance,geico’s data science team uses predictive anal...,full_time,"[geico, datum, science, team, use, predictive,..."
2,data scientist,aerospace & defense,"\ndata scientist\n \n location: washington, d...",full_time,"[datum, scientist, location, washington, dc, e..."
3,data scientist,aerospace & defense,str is a government research contractor specia...,full_time,"[str, government, research, contractor, specia..."
4,chief data scientist,business services,this company is in a hiring surge in response ...,full_time,"[company, hire, surge, response]"
...,...,...,...,...,...
3319,rn / lpn,health care,owned and operated by nurses.\nat star pediatr...,full_time,"[operate, nurse, star, pediatric, home, care, ..."
3320,ultrasound technologist,nan,medical imaging office in queens is seeking a ...,full_time,"[medical, imaging, office, queens, seek, time,..."
3321,nys licensed psychologist,nan,licensed nys psychologist needed for bronx nur...,full_time,"[license, nys, psychologist, need, bronx, nurs..."
3322,lead pharmacy technician: billing,health care,this company is in a hiring surge in response ...,full_time,"[company, hire, surge, response]"


In [ ]:
# remove non-english job posts
print("before: " + str(len(df_jobs_trimmed)))
for i, description in enumerate(df_jobs_trimmed['description_cleaned']):
  for word in description:
    if not isEnglish(word):
      df_jobs_trimmed = df_jobs_trimmed.drop(i, axis=0)
      break
print("after: " + str(len(df_jobs_trimmed)))

before: 3324
after: 3230


In [ ]:
df_jobs_trimmed.reset_index(inplace = True)
df_jobs_trimmed.drop('index', axis=1, inplace=True)
df_jobs_trimmed

,title,department,description,employment_type,description_cleaned
0,data scientist,business services,this company is in a hiring surge in response ...,full_time,"[company, hire, surge, response]"
1,data scientist,insurance,geico’s data science team uses predictive anal...,full_time,"[geico, datum, science, team, use, predictive,..."
2,data scientist,aerospace & defense,"\ndata scientist\n \n location: washington, d...",full_time,"[datum, scientist, location, washington, dc, e..."
3,data scientist,aerospace & defense,str is a government research contractor specia...,full_time,"[str, government, research, contractor, specia..."
4,chief data scientist,business services,this company is in a hiring surge in response ...,full_time,"[company, hire, surge, response]"
...,...,...,...,...,...
3225,rn / lpn,health care,owned and operated by nurses.\nat star pediatr...,full_time,"[operate, nurse, star, pediatric, home, care, ..."
3226,ultrasound technologist,nan,medical imaging office in queens is seeking a ...,full_time,"[medical, imaging, office, queens, seek, time,..."
3227,nys licensed psychologist,nan,licensed nys psychologist needed for bronx nur...,full_time,"[license, nys, psychologist, need, bronx, nurs..."
3228,lead pharmacy technician: billing,health care,this company is in a hiring surge in response ...,full_time,"[company, hire, surge, response]"


In [ ]:
for i in range(0, len(df_jobs_trimmed)):
  df_jobs_trimmed['description_cleaned'][i] = ' '.join(df_jobs_trimmed['description_cleaned'][i])

In [ ]:
df_jobs_trimmed

,title,department,description,employment_type,description_cleaned
0,data scientist,business services,this company is in a hiring surge in response ...,full_time,company hire surge response
1,data scientist,insurance,geico’s data science team uses predictive anal...,full_time,geico datum science team use predictive analyt...
2,data scientist,aerospace & defense,"\ndata scientist\n \n location: washington, d...",full_time,datum scientist location washington dc educati...
3,data scientist,aerospace & defense,str is a government research contractor specia...,full_time,str government research contractor specialize ...
4,chief data scientist,business services,this company is in a hiring surge in response ...,full_time,company hire surge response
...,...,...,...,...,...
3225,rn / lpn,health care,owned and operated by nurses.\nat star pediatr...,full_time,operate nurse star pediatric home care agency ...
3226,ultrasound technologist,nan,medical imaging office in queens is seeking a ...,full_time,medical imaging office queens seek time diagno...
3227,nys licensed psychologist,nan,licensed nys psychologist needed for bronx nur...,full_time,license nys psychologist need bronx nursing ho...
3228,lead pharmacy technician: billing,health care,this company is in a hiring surge in response ...,full_time,company hire surge response


In [ ]:
from google.colab import files
df_jobs_trimmed.to_csv('job_posts_us_city_cleaned.csv', encoding = 'utf-8-sig') 
files.download('job_posts_us_city_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Combine job posts

In [ ]:
url_fake = 'https://drive.google.com/file/d/1TQiLUY1f9bLcjBiJxiNs9yZ2TllMrT4A/view?usp=sharing'
url_fake='https://drive.google.com/uc?id=' + url_fake.split('/')[-2]

data_fake_posts = pd.read_csv(url_fake)

data_fake_posts['description_combined'] = data_fake_posts['description_cleaned'].fillna('') + data_fake_posts['requirements_cleaned'].fillna('')

In [ ]:
url_us_city = 'https://drive.google.com/file/d/1FpNhb35uY-ggBXBjqe-gVFoNmXQAdLcI/view?usp=sharing'
url_us_city='https://drive.google.com/uc?id=' + url_us_city.split('/')[-2]

data_us_city = pd.read_csv(url_us_city)

data_us_city['description_combined'] = data_us_city['description_cleaned']

In [ ]:
df_combine = [data_fake_posts, data_us_city]
df_combine = pd.concat(df_combine)
df_combine.reset_index(inplace = True)
df_combine.drop('Unnamed: 0', axis=1, inplace=True)
df_combine.drop('index', axis=1, inplace=True)
df_combine

,title,department,description,requirements,required_experience,required_education,employment_type,description_cleaned,requirements_cleaned,description_combined
0,account executive - washington dc,sales,the company: esri – environmental systems rese...,"education: bachelor’s or master’s in gis, busi...",mid-senior level,bachelor's degree,full-time,"['company', 'esri', 'environmental', 'systems'...","['education', 'bachelor', 'master', 'gis', 'bu...","['company', 'esri', 'environmental', 'systems'..."
1,bill review manager,NaN,job title: itemization review managerlocation:...,qualifications:rn license in the state of texa...,mid-senior level,bachelor's degree,full-time,"['job', 'title', 'itemization', 'review', 'man...","['qualification', 'rn', 'license', 'state', 't...","['job', 'title', 'itemization', 'review', 'man..."
2,head of content (m/f),androidpit,your responsibilities: manage the english-spea...,your know-how: ...,mid-senior level,master's degree,full-time,"['-pron-', 'responsibility', 'manage', 'englis...","['-pron-', 'know', 'university', 'college', 'd...","['-pron-', 'responsibility', 'manage', 'englis..."
3,customer service associate - part time,NaN,the customer service associate will be based i...,minimum requirements:minimum of 6 months custo...,entry level,high school or equivalent,part-time,"['customer', 'service', 'associate', 'base', '...","['minimum', 'requirement', 'minimum', 'month',...","['customer', 'service', 'associate', 'base', '..."
4,asp.net developer job opportunity at united st...,NaN,position : #url_86fd830a95a64e2b30ceed829e63fd...,position : #url_86fd830a95a64e2b30ceed829e63fd...,mid-senior level,bachelor's degree,full-time,"['position', 'developer', 'job', 'location', '...","['position', 'developer', 'job', 'location', '...","['position', 'developer', 'job', 'location', '..."
...,...,...,...,...,...,...,...,...,...,...
10668,rn / lpn,health care,owned and operated by nurses.\nat star pediatr...,NaN,NaN,NaN,full_time,operate nurse star pediatric home care agency ...,NaN,operate nurse star pediatric home care agency ...
10669,ultrasound technologist,NaN,medical imaging office in queens is seeking a ...,NaN,NaN,NaN,full_time,medical imaging office queens seek time diagno...,NaN,medical imaging office queens seek time diagno...
10670,nys licensed psychologist,NaN,licensed nys psychologist needed for bronx nur...,NaN,NaN,NaN,full_time,license nys psychologist need bronx nursing ho...,NaN,license nys psychologist need bronx nursing ho...
10671,lead pharmacy technician: billing,health care,this company is in a hiring surge in response ...,NaN,NaN,NaN,full_time,company hire surge response,NaN,company hire surge response


# export

In [ ]:
from google.colab import files
df_combine.to_csv('df_job_final.csv', encoding = 'utf-8-sig') 
files.download('df_job_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>